In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

import torch._dynamo
torch._dynamo.config.suppress_errors = True
torch._dynamo.disable()
from datetime import datetime


# choose device: GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to(device)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
torch.set_float32_matmul_precision("high")  # optional speedup
torch._dynamo.config.suppress_errors = True

start_time = datetime.now()
text = "The user said wow!"*1500 + ", what should I do?"
inputs = tokenizer(text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
probs = probs.detach().cpu()
labels = ["speak", "wait", "think", "ignore"]
predicted_label = labels[probs.argmax()]
print(predicted_label, probs)
print(f"Time taken: {datetime.now() - start_time}")


ignore tensor([[0.0097, 0.0099, 0.0267, 0.9537]])
Time taken: 0:00:00.985909
